In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import math

In [2]:
cad_df = gpd.read_file("Files/cadastre.gpkg")
gnaf_df = pd.read_parquet("Files/gnaf_prop.parquet")

In [3]:
gnaf_df['full_address'] = (
    gnaf_df['flat_number'].fillna('') + ' ' +
    gnaf_df['level_number'].fillna('') + ' ' +
    gnaf_df['building_name'].fillna('') + ' ' +
    gnaf_df['lot_number'].fillna('') + ' ' +
    gnaf_df['number_first'].fillna('') + 
    gnaf_df['number_last'].fillna('').apply(lambda x: '-' + x if x else '') + ' ' +
    gnaf_df['street_name'].fillna('') + ' ' +
    gnaf_df['street_type'].fillna('') + ' ' +
    gnaf_df['street_suffix'].fillna('') + ', ' +
    gnaf_df['locality_name'].fillna('') + ' ' +
    gnaf_df['postcode'].fillna('') + ', ' +
    gnaf_df['state'].fillna('')
)
gnaf_df['full_address'] = gnaf_df['full_address'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [6]:
import math
from shapely.geometry import Polygon, MultiPolygon

def polygon_orientation_simple(geom):
    # Pick the main polygon if it's a MultiPolygon
    if isinstance(geom, MultiPolygon):
        polygon = max(geom.geoms, key=lambda p: p.area)
    elif isinstance(geom, Polygon):
        polygon = geom
    else:
        return None  # skip anything else

    # Get first edge of the polygon
    coords = list(polygon.exterior.coords)
    p1, p2 = coords[0], coords[1]
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    
    angle = math.degrees(math.atan2(dx, dy))
    if angle < 0:
        angle += 360

    directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
    idx = int((angle + 22.5) // 45) % 8
    return directions[idx]

# Apply to cadastre
cad_df['orientation'] = cad_df['geometry'].apply(polygon_orientation_simple)

In [7]:
gnaf_gdf = gpd.GeoDataFrame(
    gnaf_df,
    geometry=gpd.points_from_xy(gnaf_df.longitude, gnaf_df.latitude),
    crs=cad_df.crs
)

In [8]:
properties = gpd.sjoin(gnaf_gdf, cad_df[['geometry','orientation']], how='left', predicate='within')


In [9]:
properties[['full_address', 'orientation']].to_csv("property_orientation.csv", index=False)

In [ ]:
print("CSV saved: property_orientation.csv")

In [10]:
print(properties['orientation'].notna().sum())
print(properties['orientation'].value_counts())


8894
orientation
S     3884
SW    1660
W     1560
E      700
SE     559
NW     202
N      183
NE     146
Name: count, dtype: int64
